In [2]:
import pandas as pd

In [5]:
data = pd.read_csv('database/counselling_data.csv')
data.shape

/tmp/ipykernel_1406936/3524630425.py:1: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('database/counselling_data.csv')


(144318, 51)

In [14]:
import pandas as pd
import json
from pathlib import Path

class TmcidJsonExporter:
    def __init__(self,
                 csv_path: str,
                 mapper_csv_path: str,
                 out_path: str = "static/question6_mapped.json"):
        self.csv_path = csv_path
        self.mapper_csv_path = mapper_csv_path
        self.out_path = out_path
        self.df = None
        self.mapper_df = None

    def load_data(self):
        # Suppress DtypeWarning by reading in low_memory mode
        self.df = pd.read_csv(self.csv_path, low_memory=False)
        self.mapper_df = pd.read_csv(self.mapper_csv_path)

    def compute_tmcid_counts(self):
        if self.df is None:
            raise RuntimeError("Data not loaded. Call load_data() first.")

        counts_series = (
            self.df
               .groupby("tmcid")
               .size()
               .sort_values(ascending=False)
        )
        actual_locs = list(counts_series.index)
        # Cast numpy.int64 to int
        counts = [int(v) for v in counts_series.values]
        return actual_locs, counts

    def to_dict(self) -> dict:
        actual_locs, counts = self.compute_tmcid_counts()

        # Build lookup: actual → mapping
        lookup = dict(zip(self.mapper_df['actual'],
                          self.mapper_df['mapping']))
        print(lookup)

        # Replace each actual tmcid with its mapped (pretty) name
        pretty_locs = [lookup.get(loc, loc) for loc in actual_locs]
        """
        for loc in actual_locs:
            print( loc,lookup.get(loc, loc) )"""

        #print(pretty_locs)

        return {
            "locations": pretty_locs,
            "values":     counts,
            "text":       pretty_locs
        }

    def save_json(self):
        data = self.to_dict()
        #mapper(data,"locations")
        out_file = Path(self.out_path)
        out_file.parent.mkdir(parents=True, exist_ok=True)
        with open(out_file, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)
        print(f"Saved mapped JSON to {out_file}")

if __name__ == "__main__":
    exporter = TmcidJsonExporter(
        csv_path="database/counselling_data.csv",
        mapper_csv_path="database/mapped_states.csv",
        out_path="static/question6_country.json"
    )
    exporter.load_data()
    exporter.save_json()



{'AFMS': 'AFMS', 'Andaman_Nicobar': 'Andaman and Nicobar Islands', 'Andhra_Pradesh': 'Andhra Pradesh', 'Arunachal_Pradesh': 'Arunachal Pradesh', 'Assam': 'Assam', 'Bihar': 'Bihar', 'Chandigarh': 'Chandigarh', 'Chhattisgarh': 'Chhattisgarh', 'Dadra_Daman_Diu': 'Dadra and Nagar Haveli', 'Delhi': 'Delhi', 'Goa': 'Goa', 'Gujarat': 'Gujarat', 'Haryana': 'Haryana', 'Himachal_Pradesh': 'Himachal Pradesh', 'Jammu_Kashmir': 'Jammu and Kashmir', 'Jharkhand': 'Jharkhand', 'Karnataka': 'Karnataka', 'Kerala': 'Kerala', 'Ladakh': 'Ladakh', 'Lakshadweep': 'Lakshadweep', 'Madhya_Pradesh': 'Madhya Pradesh', 'Maharashtra': 'Maharashtra', 'Manipur': 'Manipur', 'Meghalaya': 'Meghalaya', 'Mizoram': 'Mizoram', 'Nagaland': 'Nagaland', 'Odisha': 'Odisha', 'Pondicherry': 'Puducherry', 'Punjab': 'Punjab', 'Rajasthan': 'Rajasthan', 'Sikkim': 'Sikkim', 'Tamil_Nadu': 'Tamil Nadu', 'Telangana': 'Telangana', 'Tripura': 'Tripura', 'Uttar_Pradesh': 'Uttar Pradesh', 'Uttarakhand': 'Uttarakhand', 'West_Bengal': 'West Be

In [6]:
import pandas as  pd
def mapper(data, name): 
    mapper_data = pd.read_csv("database/mapped_states.csv")
    #print(data)

In [7]:
import pandas as pd
import json
from pathlib import Path

# Predefined order of Indian states and union territories
ORDERED_LOCATIONS = [
    "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh",
    "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand",
    "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur",
    "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab",
    "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura",
    "Uttar Pradesh", "Uttarakhand", "West Bengal", "Andaman and Nicobar Islands",
    "Chandigarh", "Dadra and Nagar Haveli and Daman and Diu", "Delhi",
    "Jammu and Kashmir", "Ladakh", "Lakshadweep", "Puducherry"
]

class TmcidJsonExporter:
    def __init__(self, csv_path: str):
        """
        Parameters
        ----------
        csv_path : str
            Path to the counselling_data.csv file.
        """
        self.csv_path = csv_path
        self.df = None

    def load_data(self):
        """Load the CSV into a pandas DataFrame."""
        self.df = pd.read_csv(self.csv_path)

    def compute_tmcid_counts(self):
        """
        Group by 'tmcid' and count rows.

        Returns
        -------
        Dict of tmcid -> count.
        """
        if self.df is None:
            raise RuntimeError("Data not loaded. Call load_data() first.")

        counts = self.df.groupby("tmcid").size().to_dict()
        return counts

    def to_dict(self) -> dict:
        """
        Build the dict for all tmcid counts in predefined order.

        Returns
        -------
        Dict matching:
        {
          "locations": [...],
          "values": [...],
          "text": [...]
        }
        """
        counts = self.compute_tmcid_counts()
        # Use fixed order, default count to 0 if missing
        locations = ORDERED_LOCATIONS.copy()
        values = [counts.get(loc, 0) for loc in locations]
        text = locations.copy()

        # If you need custom sample values instead of actual counts,
        # you can override 'values' with a fixed list as below:
        # sample_values = list(range(500, 500 * (len(locations) + 1), 100))
        # values = sample_values[:len(locations)]

        return {
            "locations": locations,
            "values": values,
            "text": text
        }

    def to_json(self) -> str:
        """Return pretty‑printed JSON string for all tmcid entries."""
        return json.dumps(self.to_dict(), indent=2)

    def save_json(self, out_path: str = "static/question6_country.json"):
        """
        Save the tmcid counts JSON into a file.

        Parameters
        ----------
        out_path : str
            File path where JSON will be written.
        """
        data_dict = self.to_dict()
        out_file = Path(out_path)
        out_file.parent.mkdir(parents=True, exist_ok=True)  # ensure directory exists
        with open(out_file, "w", encoding="utf-8") as f:
            json.dump(data_dict, f, indent=2)
        print(f"Saved JSON to {out_file}")


# ---------------- Example Usage ----------------
if __name__ == "__main__":
    exporter = TmcidJsonExporter("database/counselling_data.csv")
    exporter.load_data()
    exporter.save_json()  # writes JSON with predefined order


/tmp/ipykernel_1406936/2894724767.py:30: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(self.csv_path)


Saved JSON to static/question6_country.json


In [9]:
actual = [
    "Tamil_Nadu", "Uttar_Pradesh", "Telangana", "Odisha", "Maharashtra",
    "Karnataka", "Madhya_Pradesh", "Jammu_Kashmir", "West_Bengal", "Jharkhand",
    "Kerala", "Chhattisgarh", "Andhra_Pradesh", "Rajasthan", "Goa",
    "Dadra_Daman_Diu", "Punjab", "Tripura", "Pondicherry", "Delhi",
    "Bihar", "Assam", "Mizoram", "Gujarat", "Manipur", "Chandigarh",
    "Himachal_Pradesh", "Haryana", "Andaman_Nicobar", "Uttarakhand",
    "Ladakh", "Lakshadweep", "Nagaland", "Arunachal_Pradesh",
    "Meghalaya", "Sikkim", "AFMS"
]
df_actual = pd.DataFrame({'actual': actual})
print("Actual List DataFrame:")


ValueError: All arrays must be of the same length

In [6]:
data.columns

Index(['patient_id', 'call_id', 'ehr_id', 'triage', 'resolution',
       'created_date(counselling)', 'all_complaints', 'campaign_id', 'is_mhp',
       'segment_id', 'callendtime', 'callid', 'callstarttime', 'callstatus',
       'createdtime', 'tmcid', 'post_call â†’ transferredto',
       'post_call â†’ userid', 'customerholdtime', 'customertalktime',
       'dispositioncode', 'rating', 'call_type', 'callback_required',
       'masked_phone', 'transferredfromcampaign', 'transferredstatus',
       'transferredtocampaign', 'assessmentcalltype', 'refernce_crt_id',
       'creation_call_id', 'creator_user_id', 'Patient_masked_caller_phone',
       'patient_created_date', 'patient_year_of_birth',
       'patient â†’ campaign_id', 'patient â†’ age', 'patient â†’ restrict',
       'usertmcmapping â†’ opc_tmcid', 'usertmcmapping â†’ userid',
       'usertmcmapping â†’ tmcname', 'usertmcmapping â†’ tmcid',
       'usertmcmapping â†’ campaign_id', 'usertmcmapping â†’ statename',
       'usertmc

data1 =  pd.read_

In [10]:
import pandas as pd

# Given list
actual = [
    "Tamil_Nadu",
    "Uttar_Pradesh",
    "Telangana",
    "Odisha",
    "Maharashtra",
    "Karnataka",
    "Madhya_Pradesh",
    "Jammu_Kashmir",
    "West_Bengal",
    "Jharkhand",
    "Kerala",
    "Chhattisgarh",
    "Andhra_Pradesh",
    "Rajasthan",
    "Goa",
    "Dadra_Daman_Diu",
    "Punjab",
    "Tripura",
    "Pondicherry",
    "Delhi",
    "Bihar",
    "Assam",
    "Mizoram",
    "Gujarat",
    "Manipur",
    "Chandigarh",
    "Himachal_Pradesh",
    "Haryana",
    "Andaman_Nicobar",
    "Uttarakhand",
    "Ladakh",
    "Lakshadweep",
    "Nagaland",
    "Arunachal_Pradesh",
    "Meghalaya",
    "Sikkim",
    "AFMS"
]

# Create DataFrame
df_actual = pd.DataFrame({'actual': actual})


,actual
0,Tamil_Nadu
1,Uttar_Pradesh
2,Telangana
3,Odisha
4,Maharashtra
5,Karnataka
6,Madhya_Pradesh
7,Jammu_Kashmir
8,West_Bengal
9,Jharkhand


In [20]:
dfff = pd.DataFrame(data=df_actual['actual'].unique())


In [22]:
dfff.columns

RangeIndex(start=0, stop=1, step=1)

In [39]:
# get the name of the sole column
col = dfff.columns[0]

# return a new DataFrame sorted ascending by that column
dfff_sorted = dfff.sort_values(by=col, ascending=True)

#—or sort dfff in place:
dfff.sort_values(by=col, ascending=True, inplace=True)

dfff
df2 = dfff.copy()

In [11]:
import pandas as pd

regions = [
    "Andaman and Nicobar Islands", "Arunachal Pradesh", "Andhra Pradesh", "Assam", "Bihar", 
    "Chandigarh", "Chhattisgarh", "Dadra and Nagar Haveli", "Goa", "Gujarat", 
    "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", 
    "Lakshadweep", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", 
    "Mizoram", "Nagaland", "Delhi", "Puducherry", "Punjab", 
    "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura", 
    "Uttar Pradesh", "Uttarakhand", "West Bengal", "Odisha", 
    "Jammu and Kashmir", "Ladakh"
]

# New DataFrame
new_df = pd.DataFrame(regions, columns=['State_UT'])


In [29]:
dataframee = pd.DataFrame(data=new_df["State_UT"].unique())
col = dataframee.columns[0]

# Add "AFMS " prefix to each row
dataframee

,0
0,Andaman and Nicobar Islands
1,Arunachal Pradesh
2,Andhra Pradesh
3,Assam
4,Bihar
5,Chandigarh
6,Chhattisgarh
7,Dadra and Nagar Haveli
8,Goa
9,Gujarat


In [37]:
df1 = pd.DataFrame(data=['AFMS']+ list(dataframee[0].unique()))

In [53]:
df1.sort_values(0, inplace=True)
df1

,0
0,AFMS
1,Andaman and Nicobar Islands
3,Andhra Pradesh
2,Arunachal Pradesh
4,Assam
5,Bihar
6,Chandigarh
7,Chhattisgarh
8,Dadra and Nagar Haveli
23,Delhi


In [ ]:
df2.sort_values(0,inplace=True)


array(['AFMS', 'Andaman_Nicobar', 'Andhra_Pradesh', 'Arunachal_Pradesh',
       'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh', 'Dadra_Daman_Diu',
       'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal_Pradesh',
       'Jammu_Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh',
       'Lakshadweep', 'Madhya_Pradesh', 'Maharashtra', 'Manipur',
       'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Pondicherry',
       'Punjab', 'Rajasthan', 'Sikkim', 'Tamil_Nadu', 'Telangana',
       'Tripura', 'Uttar_Pradesh', 'Uttarakhand', 'West_Bengal'],
      dtype=object)

In [60]:
ar1 = df2[0].unique()
ar2 = df1[0].unique()
combined_df= pd.DataFrame({'mapping': ar2, 'actual':ar1})

In [62]:
combined_df.to_csv("mapped_states.csv", index=False)

In [55]:
combined_df = pd.concat([df1, df2], ignore_index=True,axis=1)
combined_df

,0,1
0,AFMS,Tamil_Nadu
1,Andaman and Nicobar Islands,Uttar_Pradesh
3,Andhra Pradesh,Odisha
2,Arunachal Pradesh,Telangana
4,Assam,Maharashtra
5,Bihar,Karnataka
6,Chandigarh,Madhya_Pradesh
7,Chhattisgarh,Jammu_Kashmir
8,Dadra and Nagar Haveli,West_Bengal
23,Delhi,Gujarat


In [ ]:

    
        #Q15 Funnale chart comparision
        df = pd.read_csv("database/Anonymized_Call_Handle_Data.csv")
        dashboard = CallFlowDashboard(dataframe=df)
        dashboard.run()



if __name__ == "__main__":
    generate_Static_JSON_files_for_dashboard_and_questions().generate()









        

Static files will be generated



/home/raghavendra/AAAA/Second Semester/PE-JAYA_MAAM/Dashboard/telemanas_dashboard/backend/dashboard.py:65: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(call_analysis_path)
/home/raghavendra/AAAA/Second Semester/PE-JAYA_MAAM/Dashboard/telemanas_dashboard/backend/dashboard.py:69: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  self.funnel_df = pd.read_csv(funnel_chart_dataset)



IN INIT


Loading and filter

Buliding for country

compute funnel

Current length:  1

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

Ending build

saving
Wrote static/callflow_by_state_funnel_chart_q1.json


/home/raghavendra/AAAA/Second Semester/PE-JAYA_MAAM/Dashboard/telemanas_dashboard/backend/dashboard.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['callenddate'] = df['callendtime'].dt.date
/home/raghavendra/AAAA/Second Semester/PE-JAYA_MAAM/Dashboard/telemanas_dashboard/backend/dashboard.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day']        = df['callenddate'].apply(lambda d: pd.to_datetime(d).day_name())
/home/raghavendra/AAAA/Second Semester/PE-JAYA_MAAM/Dashboard/telemanas_dashb

Saved: static/question2_country.json
Saved: static/question2_state.json


...

✅ Saved: static/question12_country.json and static/question12_state.json


/home/raghavendra/AAAA/Second Semester/PE-JAYA_MAAM/Dashboard/telemanas_dashboard/backend/q13_calendar.py:25: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self.csv_path)


✅ Saved to static/question13_country.json
✅ Saved to static/question13_state.json


/tmp/ipykernel_223122/2271642421.py:72: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("database/Anonymized_Call_Handle_Data.csv")



IN INIT


Loading and filter

Buliding for country

compute funnel

Current length:  1

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

compute funnel

Ending build

saving
Wrote static/callflow_by_state_funnel_chart_q1.json


In [1]:
import pandas as pd

In [5]:
data = pd.read_csv("database/sankey_care_giver.csv")
data.columns

/tmp/ipykernel_140655/1861901187.py:1: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("database/sankey_care_giver.csv")


Index(['patient_id', 'call_id', 'ehr_id', 'triage', 'resolution',
       'created_date(counselling)', 'all_complaints', 'campaign_id', 'is_mhp',
       'segment_id', 'callendtime', 'callid', 'callstarttime', 'callstatus',
       'createdtime', 'tmcid', 'post_call â†’ transferredto',
       'post_call â†’ userid', 'customerholdtime', 'customertalktime',
       'dispositioncode', 'rating', 'call_type', 'callback_required',
       'masked_phone', 'transferredfromcampaign', 'transferredstatus',
       'transferredtocampaign', 'assessmentcalltype', 'refernce_crt_id',
       'creation_call_id', 'creator_user_id', 'Patient_masked_caller_phone',
       'patient_created_date', 'patient_year_of_birth',
       'patient â†’ campaign_id', 'patient â†’ age', 'patient â†’ restrict',
       'usertmcmapping â†’ opc_tmcid', 'usertmcmapping â†’ userid',
       'usertmcmapping â†’ tmcname', 'usertmcmapping â†’ tmcid',
       'usertmcmapping â†’ campaign_id', 'usertmcmapping â†’ statename',
       'usertmc

In [7]:
data["Called_by"]

KeyError: 'Called_by'

In [116]:
import pandas as pd
data =  pd.read_csv('database/Anonymized_Call_Handle_Data.csv')


/tmp/ipykernel_51520/556679238.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data =  pd.read_csv('database/Anonymized_Call_Handle_Data.csv')


In [151]:
import pandas as pd
import geopandas as gpd
import json
import os
from typing import List, Dict, Any

class MapDataBuilder:
    def __init__(self,
                 survey_csv: str = "counselling_data.csv",
                 states_geojson: str = "INDIA_STATES.geojson",
                 output_json: str = "static/q6_map_.json"):
        self.survey_csv  = survey_csv
        self.states_geojson = states_geojson
        self.output_json = output_json

    def load_and_prepare_survey(self) -> pd.DataFrame:
        df = pd.read_csv(self.survey_csv)
        df = df[["Patient_State", "patient â†’ age"]].copy()
        df.columns = ["Patient_State", "Patient_age"]

        # fill missing
        df["Patient_age"].fillna(df["Patient_age"].mean(), inplace=True)

        # categorize
        df["Age_Group"] = df["Patient_age"].apply(
            lambda age: "Children" if age <= 20
                        else "Adults" if age <= 60
                        else "Elders"
        )

        pct = (
            df.groupby(["Patient_State", "Age_Group"]).size()
              .unstack(fill_value=0)
              .pipe(lambda d: d.div(d.sum(axis=1), axis=0) * 100)
              .reset_index()
        )
        for col in ["Children", "Adults", "Elders"]:
            if col not in pct:
                pct[col] = 0.0
        return pct

    def load_states_geo(self) -> gpd.GeoDataFrame:
        gdf = gpd.read_file(self.states_geojson)
        gdf["State_Name"] = gdf["ST_NM"].str.upper().str.strip()
        return gdf[["State_Name", "geometry"]]

    def build(self) -> List[Dict[str, Any]]:
        survey_pct = self.load_and_prepare_survey()
        gdf_states = self.load_states_geo()

        merged = pd.merge(
            survey_pct,
            gdf_states,
            left_on="Patient_State",
            right_on="State_Name",
            how="inner"
        )

        records: List[Dict[str, Any]] = []
        for _, row in merged.iterrows():
            records.append({
                "state": row["State_Name"],
                "geometry": row["geometry"].wkt,
                "Children": round(row["Children"], 6),
                "Adults":   round(row["Adults"],   6),
                "Elders":   round(row["Elders"],   6)
            })
        return records

    def save(self):
        os.makedirs(os.path.dirname(self.output_json), exist_ok=True)
        data = self.build()
        with open(self.output_json, "w") as f:
            json.dump(data, f, indent=2)
        print(f"✅ Saved map data to {self.output_json}")

    def run(self):
        self.save()


if __name__ == "__main__":
    builder = MapDataBuilder(
        survey_csv="database/counselling_data.csv",
        states_geojson="database/INDIA_STATES.geojson",
        output_json="static/q6_map_.json"
    )
    builder.run()


FileNotFoundError: [Errno 2] No such file or directory: 'database/counselling_data.csv'

In [142]:
!pip install geopandas
!pip install shapely


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [138]:
data12 =  pd.read_csv("database/counselling_data.csv")
data12.columns

/tmp/ipykernel_51520/498606730.py:1: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data12 =  pd.read_csv("database/counselling_data.csv")


Index(['patient_id', 'call_id', 'ehr_id', 'triage', 'resolution',
       'created_date(counselling)', 'all_complaints', 'campaign_id', 'is_mhp',
       'segment_id', 'callendtime', 'callid', 'callstarttime', 'callstatus',
       'createdtime', 'tmcid', 'post_call â†’ transferredto',
       'post_call â†’ userid', 'customerholdtime', 'customertalktime',
       'dispositioncode', 'rating', 'call_type', 'callback_required',
       'masked_phone', 'transferredfromcampaign', 'transferredstatus',
       'transferredtocampaign', 'assessmentcalltype', 'refernce_crt_id',
       'creation_call_id', 'creator_user_id', 'Patient_masked_caller_phone',
       'patient_created_date', 'patient_year_of_birth',
       'patient â†’ campaign_id', 'patient â†’ age', 'patient â†’ restrict',
       'usertmcmapping â†’ opc_tmcid', 'usertmcmapping â†’ userid',
       'usertmcmapping â†’ tmcname', 'usertmcmapping â†’ tmcid',
       'usertmcmapping â†’ campaign_id', 'usertmcmapping â†’ statename',
       'usertmc

In [139]:
data12.shape

(144318, 51)

In [117]:
counselling_datadata.columns

Index(['Unnamed: 0', 'crt_object_id', 'callendtime', 'callid', 'callstarttime',
       'callstatus', 'createdtime', 'tmcid', 'transferredto', 'userid',
       'customerholdtime', 'customertalktime', 'dispositioncode', 'rating',
       'call_connect', 'call_type', 'callback_required',
       'call â†’ campaign_id', 'telemanas_id', 'call â†’ incomplete_closure',
       'usertmcmapping â†’ statename', 'usertmcmapping â†’ tmcname',
       'Incomplete_closure_reason'],
      dtype='object')

In [132]:
data1 = pd.read_csv('database/counselling_complaints.csv')
data1.columns

Index(['call_id', 'user_id', 'Counsellor State', 'masked_caller_phone',
       'created_time', 'triage', 'call_types', 'segment_id',
       'counselling_data triage', 'counselling_data resolution',
       'counselling_data all_complaints', 'patient creator_user_id',
       'patient created_date', 'patient telemanas_id', 'patient age',
       'Patient District', 'Patient State', 'Patient Gender', 'callback Type',
       'Counsellor State.1', 'Called by'],
      dtype='object')

In [137]:
import pandas as pd
import json
import os
from typing import List, Dict, Any

class DistrictGenderCounter:
    def __init__(self,
                 df: pd.DataFrame,
                 state_col: str = "Patient State",
                 district_col: str = "Patient District",
                 gender_col: str = "Patient Gender",
                 output_json: str = "static/q7_district_count.json"):
        self.df = df
        self.state_col = state_col
        self.district_col = district_col
        self.gender_col = gender_col
        self.output_json = output_json

    def build(self) -> List[Dict[str, Any]]:
        result: List[Dict[str, Any]] = []

        # 1) Overall India gender counts
        gender_counts = self.df[self.gender_col].value_counts().to_dict()
        india_entry = {
            "state": "India",
            "gender_count": {
                "Male":   gender_counts.get("Male", 0),
                "Female": gender_counts.get("Female", 0)
            }
        }
        result.append(india_entry)

        # 2) Per-state, per-district counts
        for state, sdf in self.df.groupby(self.state_col):
            state_entry: Dict[str, Any] = {"state": state, "district": []}

            for district, ddf in sdf.groupby(self.district_col):
                gc = ddf[self.gender_col].value_counts().to_dict()
                district_entry = {
                    "name": district,
                    "gender_count": {
                        "Male":   gc.get("Male", 0),
                        "Female": gc.get("Female", 0)
                    }
                }
                state_entry["district"].append(district_entry)

            result.append(state_entry)

        return result

    def save(self):
        data = self.build()
        os.makedirs(os.path.dirname(self.output_json), exist_ok=True)
        with open(self.output_json, "w") as f:
            json.dump(data, f, indent=2)
        print(f"✅ Saved district/gender counts to {self.output_json}")

    def run(self):
        self.save()


# Example usage:
if __name__ == "__main__":
    df = pd.read_csv('database/counselling_complaints.csv')  # replace with your path
    builder = DistrictGenderCounter(df)
    builder.run()


✅ Saved district/gender counts to static/q7_district_count.json


In [136]:
data1['Patient Gender']

0      Female
1        Male
2      Female
3      Female
4        Male
        ...  
994      Male
995    Female
996    Female
997    Female
998    Female
Name: Patient Gender, Length: 999, dtype: object

In [134]:
data1['Patient District']

0      NARMADAPURAM
1            NAGPUR
2       MAHARAJGANJ
3         GORAKHPUR
4             GONDA
           ...     
994     MURSHIDABAD
995         CUTTACK
996         CUTTACK
997         CUTTACK
998         CUTTACK
Name: Patient District, Length: 999, dtype: object

In [120]:
import pandas as pd
import calendar
import json
import os
from typing import Dict, List

class ViolinDataBuilder:
    def __init__(self,
                 csv_path: str,
                 state_col: str = "State_Name",
                 date_col: str = "createdtime",
                 output_json: str = "static/violin_data.json"):
        self.csv_path = csv_path
        self.state_col = state_col
        self.date_col = date_col
        self.output_json = output_json
        self.df = None
        self.result: List[Dict] = []

    def load_and_clean(self):
        df = pd.read_csv(self.csv_path)

        # Rename odd column name if present
        if 'usertmcmapping â†’ statename' in df.columns:
            df = df.rename(columns={'usertmcmapping â†’ statename': self.state_col})

        # Parse dates
        df[self.date_col] = pd.to_datetime(df[self.date_col], errors="coerce")

        # Drop rows missing state or date
        df = df.dropna(subset=[self.state_col, self.date_col])

        # Normalize state strings
        if pd.api.types.is_string_dtype(df[self.state_col]):
            df[self.state_col] = df[self.state_col].str.strip().str.title()

        self.df = df

    @staticmethod
    def month_end_start_middle_values(subdf: pd.DataFrame, date_col: str) -> Dict[str, List[int]]:
        """
        Counts events per day, then splits into:
          - beg_end (days 1–5, 25+)
          - middle (days 6–24)
        """
        counts = subdf[date_col].dt.day.value_counts().sort_index()

        beg_end = []
        middle  = []
        for day, cnt in counts.items():
            if day <= 5 or day >= 25:
                beg_end.append(int(cnt))
            else:
                middle.append(int(cnt))

        return {"beg_end": beg_end, "middle": middle}

    def build(self):
        # --- First: the whole‐country ("India") entry ---
        india_entry = {"state": "India", "months": []}
        for period, mdf in self.df.groupby(self.df[self.date_col].dt.to_period("M")):
            vals = self.month_end_start_middle_values(mdf, self.date_col)
            month_name = calendar.month_name[period.month]
            india_entry["months"].append({
                "Month": month_name,
                "data": vals
            })
        india_entry["months"].sort(
            key=lambda x: list(calendar.month_name).index(x["Month"])
        )
        self.result.append(india_entry)

        # --- Then: each individual state ---
        for state, sdf in self.df.groupby(self.state_col):
            entry = {"state": state, "months": []}
            for period, mdf in sdf.groupby(sdf[self.date_col].dt.to_period("M")):
                vals = self.month_end_start_middle_values(mdf, self.date_col)
                month_name = calendar.month_name[period.month]
                entry["months"].append({
                    "Month": month_name,
                    "data": vals
                })
            entry["months"].sort(
                key=lambda x: list(calendar.month_name).index(x["Month"])
            )
            self.result.append(entry)

    def save(self):
        os.makedirs(os.path.dirname(self.output_json), exist_ok=True)
        with open(self.output_json, "w") as f:
            json.dump(self.result, f, indent=2)
        print(f"✅ Violin data saved to {self.output_json}")

    def run(self):
        self.load_and_clean()
        self.build()
        self.save()


if __name__ == "__main__":
    builder = ViolinDataBuilder(
        csv_path="database/Anonymized_Call_Handle_Data.csv",
        state_col="State_Name",
        date_col="createdtime",
        output_json="static/violin_data.json"
    )
    builder.run()


/tmp/ipykernel_51520/422141920.py:21: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self.csv_path)


✅ Violin data saved to static/violin_data.json


In [121]:
data.columns

Index(['Unnamed: 0', 'crt_object_id', 'callendtime', 'callid', 'callstarttime',
       'callstatus', 'createdtime', 'tmcid', 'transferredto', 'userid',
       'customerholdtime', 'customertalktime', 'dispositioncode', 'rating',
       'call_connect', 'call_type', 'callback_required',
       'call â†’ campaign_id', 'telemanas_id', 'call â†’ incomplete_closure',
       'usertmcmapping â†’ statename', 'usertmcmapping â†’ tmcname',
       'Incomplete_closure_reason'],
      dtype='object')

In [126]:
{ 
    
window_1 =  5:00 - 8:50
window_2 =  9:00 - 11:59
window_3 =  12:00 - 15:59
window_4 =  16:00 - 20:30
window_5 =  20:31 - 23:59
window_6 =  00:00 - 4:59 

} 

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (556314009.py, line 1)

In [123]:
data['createdtime']

0          2024-05-25T13:36:21.293
1          2024-05-27T07:09:23.702
2          2024-05-28T14:02:39.755
3          2024-05-28T14:02:39.694
4          2024-05-28T14:02:39.755
                    ...           
1171964    2024-07-24T19:06:54.297
1171965     2024-09-21T09:12:02.72
1171966    2024-07-25T16:28:14.232
1171967    2024-08-19T19:02:28.095
1171968    2024-08-19T19:02:38.217
Name: createdtime, Length: 1171969, dtype: object

In [128]:
import pandas as pd
import json
import os
from datetime import time
from typing import List, Dict, Any

class TimeWindowCounter:
    def __init__(self,
                 csv_path: str,
                 tmcid_col: str = "tmcid",
                 time_col: str = "createdtime",
                 output_json: str = "static/timings.json"):
        self.csv_path = csv_path
        self.tmcid_col = tmcid_col
        self.time_col = time_col
        self.output_json = output_json
        self.df: pd.DataFrame = None

        # Define windows as (label, start_time, end_time)
        self.windows = [
            ("5:00 - 8:59",   time(5, 0),  time(8, 59, 59)),
            ("9:00 - 11:59",  time(9, 0),  time(11, 59, 59)),
            ("12:00 - 15:59", time(12, 0), time(15, 59, 59)),
            ("16:00 - 20:30", time(16, 0), time(20, 30, 0)),
            ("20:31 - 23:59", time(20, 31), time(23, 59, 59)),
            ("00:00 - 04:59", time(0, 0),  time(4, 59, 59)),
        ]

    def load_and_parse(self):
        df = pd.read_csv(self.csv_path)
        df[self.time_col] = pd.to_datetime(df[self.time_col], errors="coerce")
        df = df.dropna(subset=[self.tmcid_col, self.time_col])
        self.df = df

    def assign_window(self, t: time) -> str:
        for label, start, end in self.windows:
            if start <= end:
                if start <= t <= end:
                    return label
            else:
                if t >= start or t <= end:
                    return label
        return "Unknown"

    def count_per_tmc(self) -> List[Dict[str, Any]]:
        results: List[Dict[str, Any]] = []

        # 1) Compute overall ("India") counts
        self.df["window"] = self.df[self.time_col].dt.time.apply(self.assign_window)
        overall_counts = self.df["window"].value_counts().to_dict()
        overall_timing = []
        for label, _, _ in self.windows:
            overall_timing.append({label: overall_counts.get(label, 0)})
        results.append({"state": "India", "timings": overall_timing})

        # 2) Then per-tmcid
        for tmc, sub in self.df.groupby(self.tmcid_col):
            counts = sub["window"].value_counts().to_dict()
            timing_list = []
            for label, _, _ in self.windows:
                timing_list.append({label: counts.get(label, 0)})
            results.append({"state": str(tmc), "timings": timing_list})

        return results

    def save(self):
        out = self.count_per_tmc()
        os.makedirs(os.path.dirname(self.output_json), exist_ok=True)
        with open(self.output_json, "w") as f:
            json.dump(out, f, indent=2)
        print(f"✅ Saved timing counts to {self.output_json}")

    def run(self):
        self.load_and_parse()
        self.save()


if __name__ == "__main__":
    builder = TimeWindowCounter(
        csv_path="database/Anonymized_Call_Handle_Data.csv",
        tmcid_col="tmcid",
        time_col="createdtime",
        output_json="static/timings.json"
    )
    builder.run()


/tmp/ipykernel_51520/3299598764.py:30: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self.csv_path)


✅ Saved timing counts to static/timings.json


In [130]:
import pandas as pd
import json
import os
from typing import List, Dict, Any

class CalendarDataBuilder:
    def __init__(self,
                 csv_path: str,
                 tmcid_col: str = "tmcid",
                 time_col: str = "createdtime",
                 output_json: str = "static/q5_calendar.json"):
        self.csv_path = csv_path
        self.tmcid_col = tmcid_col
        self.time_col = time_col
        self.output_json = output_json
        self.df: pd.DataFrame = None

    def load_and_parse(self):
        # Load CSV and parse timestamps
        df = pd.read_csv(self.csv_path)
        df[self.time_col] = pd.to_datetime(df[self.time_col], errors="coerce")
        df = df.dropna(subset=[self.tmcid_col, self.time_col])
        # Extract date-only string
        df["date_only"] = df[self.time_col].dt.date.astype(str)
        self.df = df

    def build(self) -> List[Dict[str, Any]]:
        calendar_list: List[Dict[str, Any]] = []

        # 1) Overall country-level counts as "India"
        overall_counts = (
            self.df["date_only"]
            .value_counts()
            .sort_index()
            .to_dict()
        )
        overall_date_counts = [
            {"date": date, "count": count}
            for date, count in overall_counts.items()
        ]
        calendar_list.append({
            "state": "India",
            "date_counts": overall_date_counts
        })

        # 2) Per-TMCID counts
        for tmc, subdf in self.df.groupby(self.tmcid_col):
            counts = (
                subdf["date_only"]
                .value_counts()
                .sort_index()
                .to_dict()
            )
            date_counts = [
                {"date": date, "count": count}
                for date, count in counts.items()
            ]
            calendar_list.append({
                "state": str(tmc),
                "date_counts": date_counts
            })

        return calendar_list

    def save(self):
        # Build data and write to JSON
        data = self.build()
        os.makedirs(os.path.dirname(self.output_json), exist_ok=True)
        with open(self.output_json, "w") as f:
            json.dump(data, f, indent=2)
        print(f"✅ Calendar data saved to {self.output_json}")

    def run(self):
        self.load_and_parse()
        self.save()


if __name__ == "__main__":
    builder = CalendarDataBuilder(
        csv_path="database/Anonymized_Call_Handle_Data.csv",
        tmcid_col="tmcid",
        time_col="createdtime",
        output_json="static/q5_calendar.json"
    )
    builder.run()


/tmp/ipykernel_51520/2803835298.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self.csv_path)


✅ Calendar data saved to static/q5_calendar.json


In [4]:
import os
import pandas as pd

'\n"state": "Karnataka",\n    "totalCalls": 2100,\n    "byGender": { "Male": 1200, "Female": 900 },\n    "timeseries": [\n      { "date": "2025-01-01", "calls": 50 },\n      { "date": "2025-01-02", "calls": 55 },\n      { "date": "2025-01-03", "calls": 60 },\n      { "date": "2025-01-04", "calls": 65 },\n      { "date": "2025-01-05", "calls": 70 },\n      { "date": "2025-01-06", "calls": 75 },\n      { "date": "2025-01-07", "calls": 80 }\n    ],\n    "avgDuration": [\n      { "date": "2025-01-01", "minutes": 2.2 },\n      { "date": "2025-01-02", "minutes": 2.4 },\n      { "date": "2025-01-03", "minutes": 2.6 },\n      { "date": "2025-01-04", "minutes": 2.8 },\n      { "date": "2025-01-05", "minutes": 3.0 },\n      { "date": "2025-01-06", "minutes": 3.2 },\n      { "date": "2025-01-07", "minutes": 3.4 }\n    ],\n    "byCategory": {\n      "Anxiety": 700,\n      "Depression": 600,\n      "Stress": 400,\n      "Relationship": 250,\n      "Other": 150\n    }\n'

In [5]:
os.listdir("database")

['info.txt',
 'Anonymized_Call_Handle_Data.csv',
 'repeated_calls.csv',
 'Call_not_picked.csv',
 'call_connected_issues.csv',
 'counselling_data.csv',
 'Telemana_Analysis.csv',
 'counselling_complaints.csv']

In [7]:
for i in os.listdir("database"):
    pth = os.path.join("database", i)
    data = pd.read_csv(pth)
    print("\n\n\n", pth, "\n", data.columns,"\n",data.shape)





 database/info.txt 
 Index(['CNCL  = counselling.csv'], dtype='object') 
 (0, 1)


/tmp/ipykernel_59082/1831445567.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(pth)





 database/Anonymized_Call_Handle_Data.csv 
 Index(['Unnamed: 0', 'crt_object_id', 'callendtime', 'callid', 'callstarttime',
       'callstatus', 'createdtime', 'tmcid', 'transferredto', 'userid',
       'customerholdtime', 'customertalktime', 'dispositioncode', 'rating',
       'call_connect', 'call_type', 'callback_required',
       'call â†’ campaign_id', 'telemanas_id', 'call â†’ incomplete_closure',
       'usertmcmapping â†’ statename', 'usertmcmapping â†’ tmcname',
       'Incomplete_closure_reason'],
      dtype='object') 
 (1171969, 23)


/tmp/ipykernel_59082/1831445567.py:3: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(pth)





 database/repeated_calls.csv 
 Index(['crt_object_id', 'callendtime', 'callid', 'callstarttime', 'callstatus',
       'createdtime', 'tmcid', 'transferredto', 'userid', 'customerholdtime',
       'customertalktime', 'dispositioncode', 'rating', 'call_connect',
       'call_type', 'callback_required', 'masked_phone',
       'call - crt_object_id â†’ queue_language',
       'call - crt_object_id â†’ created_time',
       'call - crt_object_id â†’ callback_type',
       'call - crt_object_id â†’ triage', 'telemanas_id',
       'call - crt_object_id â†’ call_types',
       'call - crt_object_id â†’ incomplete_closure',
       'call - crt_object_id â†’ segment_id',
       'call - crt_object_id â†’ reference_id',
       'call - crt_object_id â†’ call_unsuccess_reason',
       'call - crt_object_id â†’ prank_type',
       'call - crt_object_id â†’ prank_id',
       'call - crt_object_id â†’ incomplete_closure_desc',
       'usertmcmapping â†’ tmcname', 'usertmcmapping â†’ statename',
     

/tmp/ipykernel_59082/1831445567.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(pth)





 database/call_connected_issues.csv 
 Index(['Unnamed: 0.1', 'Unnamed: 0', 'crt_object_id', 'callendtime', 'callid',
       'callstarttime', 'callstatus', 'createdtime', 'tmcid', 'transferredto',
       'userid', 'customerholdtime', 'customertalktime', 'dispositioncode',
       'rating', 'call_connect', 'call_type', 'callback_required',
       'call â†’ campaign_id', 'telemanas_id', 'call â†’ incomplete_closure',
       'usertmcmapping â†’ statename', 'usertmcmapping â†’ tmcname',
       'Incomplete_closure_reason'],
      dtype='object') 
 (791656, 24)


/tmp/ipykernel_59082/1831445567.py:3: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(pth)





 database/counselling_data.csv 
 Index(['patient_id', 'call_id', 'ehr_id', 'triage', 'resolution',
       'created_date(counselling)', 'all_complaints', 'campaign_id', 'is_mhp',
       'segment_id', 'callendtime', 'callid', 'callstarttime', 'callstatus',
       'createdtime', 'tmcid', 'post_call â†’ transferredto',
       'post_call â†’ userid', 'customerholdtime', 'customertalktime',
       'dispositioncode', 'rating', 'call_type', 'callback_required',
       'masked_phone', 'transferredfromcampaign', 'transferredstatus',
       'transferredtocampaign', 'assessmentcalltype', 'refernce_crt_id',
       'creation_call_id', 'creator_user_id', 'Patient_masked_caller_phone',
       'patient_created_date', 'patient_year_of_birth',
       'patient â†’ campaign_id', 'patient â†’ age', 'patient â†’ restrict',
       'usertmcmapping â†’ opc_tmcid', 'usertmcmapping â†’ userid',
       'usertmcmapping â†’ tmcname', 'usertmcmapping â†’ tmcid',
       'usertmcmapping â†’ campaign_id', 'usertmcmap

In [ ]:
"""
Gender column is present in
1. Telemana_Analysis.csv (patient - gender)(999)
2. counselling_data.csv  [Gender - 144318]
3. counselling_complaints.csv (patient - gender)(999)
"""

In [2]:
import pandas as pd

In [3]:
d1 =  pd.read_csv("database/counselling_data.csv")

/tmp/ipykernel_51520/1260520627.py:1: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  d1 =  pd.read_csv("database/counselling_data.csv")


In [4]:
d1.columns

Index(['patient_id', 'call_id', 'ehr_id', 'triage', 'resolution',
       'created_date(counselling)', 'all_complaints', 'campaign_id', 'is_mhp',
       'segment_id', 'callendtime', 'callid', 'callstarttime', 'callstatus',
       'createdtime', 'tmcid', 'post_call â†’ transferredto',
       'post_call â†’ userid', 'customerholdtime', 'customertalktime',
       'dispositioncode', 'rating', 'call_type', 'callback_required',
       'masked_phone', 'transferredfromcampaign', 'transferredstatus',
       'transferredtocampaign', 'assessmentcalltype', 'refernce_crt_id',
       'creation_call_id', 'creator_user_id', 'Patient_masked_caller_phone',
       'patient_created_date', 'patient_year_of_birth',
       'patient â†’ campaign_id', 'patient â†’ age', 'patient â†’ restrict',
       'usertmcmapping â†’ opc_tmcid', 'usertmcmapping â†’ userid',
       'usertmcmapping â†’ tmcname', 'usertmcmapping â†’ tmcid',
       'usertmcmapping â†’ campaign_id', 'usertmcmapping â†’ statename',
       'usertmc

In [6]:
d1.shape

(144318, 51)

In [ ]:
d1.

In [14]:
li = d1['Patient_State'].unique()

array(['KARNATAKA', 'TRIPURA', 'TELANGANA', 'BIHAR', 'UTTAR PRADESH',
       'ODISHA', 'JAMMU AND KASHMIR', 'TAMIL NADU', 'MAHARASHTRA',
       'MADHYA PRADESH', 'HARYANA', 'WEST BENGAL', 'JHARKHAND', nan,
       'KERALA', 'ASSAM', 'PUNJAB', 'GUJARAT', 'ANDHRA PRADESH',
       'RAJASTHAN', 'GOA', 'CHHATTISGARH',
       'DADRA AND NAGAR HAVELI AND DAMAN AND DIU', 'UTTARAKHAND',
       'LAKSHADWEEP', 'HIMACHAL PRADESH', 'PUDUCHERRY', 'MANIPUR',
       'DELHI', 'CHANDIGARH', 'MIZORAM', 'ANDAMAN AND NICOBAR ISLANDS',
       'ARUNACHAL PRADESH', 'LADAKH', 'NAGALAND', 'SIKKIM', 'MEGHALAYA'],
      dtype=object)

In [20]:
keep = {'Male', 'Female', 'Transgender'}
print(d1.shape)
print(d1['Gender'].unique())

# Filter
d1 = d1[d1['Gender'].isin(keep)].reset_index(drop=True)
print(d1.shape)
d1['Gender'].unique()

(144318, 51)
['Male' 'Female' nan 'Prefer Not To Say' 'Other' 'Transgender']
(142559, 51)


array(['Male', 'Female', 'Transgender'], dtype=object)

In [22]:
d2=d1[d1['Patient_State']=="KARNATAKA"]

In [33]:

dic = {}
dic['state']="KARNATAKA"
dic['totalCalls']=d2.shape[0]

d2.columns

Index(['patient_id', 'call_id', 'ehr_id', 'triage', 'resolution',
       'created_date(counselling)', 'all_complaints', 'campaign_id', 'is_mhp',
       'segment_id', 'callendtime', 'callid', 'callstarttime', 'callstatus',
       'createdtime', 'tmcid', 'post_call â†’ transferredto',
       'post_call â†’ userid', 'customerholdtime', 'customertalktime',
       'dispositioncode', 'rating', 'call_type', 'callback_required',
       'masked_phone', 'transferredfromcampaign', 'transferredstatus',
       'transferredtocampaign', 'assessmentcalltype', 'refernce_crt_id',
       'creation_call_id', 'creator_user_id', 'Patient_masked_caller_phone',
       'patient_created_date', 'patient_year_of_birth',
       'patient â†’ campaign_id', 'patient â†’ age', 'patient â†’ restrict',
       'usertmcmapping â†’ opc_tmcid', 'usertmcmapping â†’ userid',
       'usertmcmapping â†’ tmcname', 'usertmcmapping â†’ tmcid',
       'usertmcmapping â†’ campaign_id', 'usertmcmapping â†’ statename',
       'usertmc

In [31]:
d1['Gender'].unique()

array(['Male', 'Female', 'Transgender'], dtype=object)

In [34]:
gen={}
gen["Male"] = d2[d2['Gender']=='Male'].shape[0]
gen["Female"] = d2[d2['Gender']=='Female'].shape[0]
gen['Transgender'] = d2[d2['Gender']=='Transgender'].shape[0]

dic["byGender"]= gen
dic


{'state': 'KARNATAKA',
 'totalCalls': 9201,
 'byGender': {'Male': 5122, 'Female': 4079, 'Transgender': 0}}

In [55]:
d1.columns

Index(['patient_id', 'call_id', 'ehr_id', 'triage', 'resolution',
       'created_date(counselling)', 'all_complaints', 'campaign_id', 'is_mhp',
       'segment_id', 'callendtime', 'callid', 'callstarttime', 'callstatus',
       'createdtime', 'tmcid', 'post_call â†’ transferredto',
       'post_call â†’ userid', 'customerholdtime', 'customertalktime',
       'dispositioncode', 'rating', 'call_type', 'callback_required',
       'masked_phone', 'transferredfromcampaign', 'transferredstatus',
       'transferredtocampaign', 'assessmentcalltype', 'refernce_crt_id',
       'creation_call_id', 'creator_user_id', 'Patient_masked_caller_phone',
       'patient_created_date', 'patient_year_of_birth',
       'patient â†’ campaign_id', 'patient â†’ age', 'patient â†’ restrict',
       'usertmcmapping â†’ opc_tmcid', 'usertmcmapping â†’ userid',
       'usertmcmapping â†’ tmcname', 'usertmcmapping â†’ tmcid',
       'usertmcmapping â†’ campaign_id', 'usertmcmapping â†’ statename',
       'usertmc

In [12]:
# Convert to datetime
d1['callendtime'] = pd.to_datetime(d1['callendtime'], errors='coerce')
d1['callstarttime'] = pd.to_datetime(d1['callstarttime'], errors='coerce')

# Drop rows with invalid datetimes
d1 = d1.dropna(subset=['callendtime', 'callstarttime'])

# Extract date
d1['callenddate'] =  pd.to_datetime(d1['callendtime'].dt.date)

# Extract time from datetime
d1['callendtimeframe'] =  d1['callendtime'].dt.time
d1['callstarttimeframe'] =  d1['callstarttime'].dt.time




def time_to_minutes(t):
    return t.hour * 60 + t.minute + t.second / 60

# Apply conversion
d1['callend_minutes'] =  d1['callendtimeframe'].apply(time_to_minutes)
d1['callstart_minutes'] =  d1['callstarttimeframe'].apply(time_to_minutes)

# Compute the difference in minutes
d1['call_duration_minutes_from_time'] =  d1['callend_minutes'] - d1['callstart_minutes']


In [13]:
d1['callenddate']

0        2024-10-07
1        2024-10-07
2        2024-10-07
3        2024-10-07
4        2024-10-07
            ...    
144313   2024-10-07
144314   2024-10-07
144315   2024-10-07
144316   2024-10-07
144317   2024-10-07
Name: callenddate, Length: 144317, dtype: datetime64[ns]

In [ ]:
d1['day'] = d1['callenddate'].dt.day_name()



In [17]:
d1['day']

0         Monday
1         Monday
2         Monday
3         Monday
4         Monday
           ...  
144313    Monday
144314    Monday
144315    Monday
144316    Monday
144317    Monday
Name: day, Length: 144317, dtype: object

In [68]:
sum(d1['call_duration_minutes_from_time'])/d1.shape[0]

4.632668223927681

In [75]:
def avg_duration(df):
    temp_l = []
    for i, j in df.groupby('callenddate'): 
        temp = {}
        temp["date"] = str(i)
        avg = sum(j['call_duration_minutes_from_time']) / j.shape[0]
        temp["minutes"] = round(avg, 2)  # Correct use of round()
        temp_l.append(temp)

    return temp_l

avg_duration(d1)

    


[{'date': '2024-10-04', 'minutes': 8.68},
 {'date': '2024-10-05', 'minutes': 2.45},
 {'date': '2024-10-06', 'minutes': 6.45},
 {'date': '2024-10-07', 'minutes': 6.53},
 {'date': '2024-10-08', 'minutes': 2.63},
 {'date': '2024-10-09', 'minutes': 5.44},
 {'date': '2024-10-10', 'minutes': 7.75},
 {'date': '2024-10-11', 'minutes': 4.29},
 {'date': '2024-10-12', 'minutes': 8.8},
 {'date': '2024-10-13', 'minutes': 2.64},
 {'date': '2024-10-14', 'minutes': 7.13},
 {'date': '2024-10-15', 'minutes': 3.98},
 {'date': '2024-10-16', 'minutes': 2.55},
 {'date': '2024-10-17', 'minutes': 5.25},
 {'date': '2024-10-18', 'minutes': 6.38},
 {'date': '2024-10-19', 'minutes': 5.14},
 {'date': '2024-10-20', 'minutes': 3.39},
 {'date': '2024-10-21', 'minutes': 2.17},
 {'date': '2024-10-22', 'minutes': 6.2},
 {'date': '2024-10-23', 'minutes': 7.88},
 {'date': '2024-10-24', 'minutes': 6.0},
 {'date': '2024-10-25', 'minutes': 3.94},
 {'date': '2024-10-26', 'minutes': 2.78},
 {'date': '2024-10-27', 'minutes': 4.

In [77]:
d1['tmcid'].unique()

array(['Karnataka', 'Tripura', 'Telangana', 'Bihar', 'Uttar_Pradesh',
       'Odisha', 'Jammu_Kashmir', 'Tamil_Nadu', 'Maharashtra',
       'Madhya_Pradesh', 'Haryana', 'West_Bengal', 'Jharkhand', 'Kerala',
       'Assam', 'Chhattisgarh', 'Punjab', 'Dadra_Daman_Diu',
       'Andaman_Nicobar', 'Andhra_Pradesh', 'Rajasthan', 'Pondicherry',
       'Chandigarh', 'Goa', 'Himachal_Pradesh', 'Manipur', 'Delhi',
       'Gujarat', 'Mizoram', 'Lakshadweep', 'Uttarakhand', 'Ladakh',
       'Meghalaya', 'AFMS', 'Arunachal_Pradesh', 'Nagaland', 'Sikkim'],
      dtype=object)

In [ ]:
pd.read_csv()

KeyError: ('callendtimeframe', 'callstarttimeframe')

In [43]:
import pandas as pd
def group_by_age(dataframe):
   

    # Drop NaN values in the age column
    age_col = "patient â†’ age"
    df = dataframe.dropna(subset=[age_col])

    # Convert to numeric, coercing any non-numeric to NaN (then drop them)
    df[age_col] = pd.to_numeric(df[age_col], errors='coerce')
    df = df.dropna(subset=[age_col])

    # Define bins and labels
    bins = [0, 18, 25, 35, 45, 55, float('inf')]
    labels = ["Under 18", "18-24", "25-34", "35-44", "45-54", "55+"]

    # Cut ages into bins
    df['age_group'] = pd.cut(df[age_col], bins=bins, labels=labels, right=False)

    # Count values per age group
    age_counts = df['age_group'].value_counts().reindex(labels, fill_value=0)

    # Convert to dictionary
    return age_counts.to_dict()


group_by_age(d1)


{'Under 18': 8915,
 '18-24': 27930,
 '25-34': 43644,
 '35-44': 26795,
 '45-54': 17100,
 '55+': 11771}

In [39]:
d1 = d1.dropna(subset=["patient â†’ age"])

In [42]:
d1["callenddate"]

0        2024-10-07
1        2024-10-07
2        2024-10-07
3        2024-10-07
4        2024-10-07
            ...    
144313   2024-10-07
144314   2024-10-07
144315   2024-10-07
144316   2024-10-07
144317   2024-10-07
Name: callenddate, Length: 136155, dtype: datetime64[ns]

In [46]:
d1['call_type'].unique()

array(['Incoming', 'Outgoing'], dtype=object)

In [44]:
d1['Month'] = d1['callenddate'].dt.strftime('%b')
d1['Month']

0         Oct
1         Oct
2         Oct
3         Oct
4         Oct
         ... 
144313    Oct
144314    Oct
144315    Oct
144316    Oct
144317    Oct
Name: Month, Length: 136155, dtype: object

In [47]:
def give_call_count(dataframe, month): 
    temp={}
    temp['month'] = month
    for i, j in dataframe.groupby('call_type'):
        temp[i.lower()] =  j.shape[0]

    return temp

def callsByDirection(dataframe):
    temp_l=[]
    for i, j in dataframe.groupby('Month'):
        temp_l.append(give_call_count(j,i))

    return temp_l

callsByDirection(d1)

    



[{'month': 'Dec', 'incoming': 34154, 'outgoing': 12934},
 {'month': 'Nov', 'incoming': 32335, 'outgoing': 11498},
 {'month': 'Oct', 'incoming': 34375, 'outgoing': 10859}]

In [53]:
def traiage(dataframe):
    temp={}
    for i, j in dataframe.groupby('triage'):
        temp[i]=  j.shape[0]

    return temp

traiage(d1)
    

    {'Emergency': 979, 'Information': 24959, 'Prank': 3237, 'Routine': 106954}
    

{'Emergency': 979, 'Information': 24959, 'Prank': 3237, 'Routine': 106954}

In [56]:
data =  pd.read_csv('database/counselling_complaints.csv')
data.columns

Index(['call_id', 'user_id', 'Counsellor State', 'masked_caller_phone',
       'created_time', 'triage', 'call_types', 'segment_id',
       'counselling_data triage', 'counselling_data resolution',
       'counselling_data all_complaints', 'patient creator_user_id',
       'patient created_date', 'patient telemanas_id', 'patient age',
       'Patient District', 'Patient State', 'Patient Gender', 'callback Type',
       'Counsellor State.1', 'Called by'],
      dtype='object')

In [58]:
d1['tmcid'].unique()

array(['Karnataka', 'Tripura', 'Telangana', 'Bihar', 'Uttar_Pradesh',
       'Odisha', 'Jammu_Kashmir', 'Tamil_Nadu', 'Maharashtra',
       'Madhya_Pradesh', 'Haryana', 'West_Bengal', 'Jharkhand', 'Kerala',
       'Assam', 'Chhattisgarh', 'Punjab', 'Dadra_Daman_Diu',
       'Andaman_Nicobar', 'Andhra_Pradesh', 'Rajasthan', 'Pondicherry',
       'Chandigarh', 'Goa', 'Himachal_Pradesh', 'Manipur', 'Delhi',
       'Gujarat', 'Mizoram', 'Lakshadweep', 'Uttarakhand', 'Ladakh',
       'Meghalaya', 'AFMS', 'Arunachal_Pradesh', 'Nagaland', 'Sikkim'],
      dtype=object)

In [60]:
data['Counsellor State'].unique()

array(['Madhya_Pradesh', 'Maharashtra', 'Uttar_Pradesh', 'Jharkhand',
       'Karnataka', 'Tripura', 'Telangana', 'Assam', 'Odisha',
       'Tamil_Nadu', 'Haryana', 'Kerala', 'Andhra_Pradesh', 'West_Bengal',
       'Rajasthan', 'Bihar', 'Chhattisgarh', 'Jammu_Kashmir', 'Punjab',
       'Delhi', 'Pondicherry', 'Goa', 'Manipur', 'Dadra_Daman_Diu',
       'Gujarat', 'Mizoram', 'Chandigarh'], dtype=object)

In [64]:
d1['tmcid'].unique()

array(['Karnataka', 'Tripura', 'Telangana', 'Bihar', 'Uttar_Pradesh',
       'Odisha', 'Jammu_Kashmir', 'Tamil_Nadu', 'Maharashtra',
       'Madhya_Pradesh', 'Haryana', 'West_Bengal', 'Jharkhand', 'Kerala',
       'Assam', 'Chhattisgarh', 'Punjab', 'Dadra_Daman_Diu',
       'Andaman_Nicobar', 'Andhra_Pradesh', 'Rajasthan', 'Pondicherry',
       'Chandigarh', 'Goa', 'Himachal_Pradesh', 'Manipur', 'Delhi',
       'Gujarat', 'Mizoram', 'Lakshadweep', 'Uttarakhand', 'Ladakh',
       'Meghalaya', 'AFMS', 'Arunachal_Pradesh', 'Nagaland', 'Sikkim'],
      dtype=object)

In [75]:
d1.columns

Index(['patient_id', 'call_id', 'ehr_id', 'triage', 'resolution',
       'created_date(counselling)', 'all_complaints', 'campaign_id', 'is_mhp',
       'segment_id', 'callendtime', 'callid', 'callstarttime', 'callstatus',
       'createdtime', 'tmcid', 'post_call â†’ transferredto',
       'post_call â†’ userid', 'customerholdtime', 'customertalktime',
       'dispositioncode', 'rating', 'call_type', 'callback_required',
       'masked_phone', 'transferredfromcampaign', 'transferredstatus',
       'transferredtocampaign', 'assessmentcalltype', 'refernce_crt_id',
       'creation_call_id', 'creator_user_id', 'Patient_masked_caller_phone',
       'patient_created_date', 'patient_year_of_birth',
       'patient â†’ campaign_id', 'patient â†’ age', 'patient â†’ restrict',
       'usertmcmapping â†’ opc_tmcid', 'usertmcmapping â†’ userid',
       'usertmcmapping â†’ tmcname', 'usertmcmapping â†’ tmcid',
       'usertmcmapping â†’ campaign_id', 'usertmcmapping â†’ statename',
       'usertmc

In [76]:
d1['customertalktime']

0         00:05:58
1         00:48:40
2         00:03:21
3         00:02:22
4         00:05:13
            ...   
144313    00:12:58
144314    00:05:04
144315    00:47:41
144316    00:47:41
144317    00:18:13
Name: customertalktime, Length: 136155, dtype: object

/tmp/ipykernel_51520/454076741.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d1['customertalktime'] =  pd.to_datetime(d1['customertalktime'])


In [87]:
# assume d1['customertalktime'] is dtype datetime64[ns], e.g. "2025-01-01 00:03:25"
# normalize() gives midnight of each date
midnight = d1['customertalktime'].dt.normalize()
# subtract to get a Timedelta (duration since midnight)
d1['customertalktime'] = d1['customertalktime'] - midnight

# now it's a timedelta series, so you can average:
result = [
    {
        "date": str(date),
        "minutes": round(group['customertalktime'].mean().total_seconds() / 60, 2)
    }
    for date, group in d1.groupby('callenddate')
]


AttributeError: 'TimedeltaProperties' object has no attribute 'normalize'

In [89]:
resultresult = [
    {
        "date": date.strftime('%Y-%m-%d'),  # Format the date without time
        "minutes": round(group['customertalktime'].mean().total_seconds() / 60, 2)
    }
    for date, group in d1.groupby('callenddate')
]

In [84]:
d1['customertalktime'] =  pd.to_timedelta(d1['customertalktime'])
[{
            "date": str(i),
            "minutes": round(sum(j['customertalktime']) / j.shape[0], 2)
        } for i, j in d1.groupby('callenddate')]





TypeError: dtype datetime64[ns] cannot be converted to timedelta64[ns]

In [100]:
import pandas as pd
import numpy as np
df = pd.read_csv("database/Anonymized_Call_Handle_Data.csv")
df['call - crt_object_id â†’ masked_caller_phone'].unique().shape[0]
df['customertalktime'].head()
values_to_remove = ['ML02_TMC','docutoroutboud','KIRAN','IIITB_OB','Training_TMC_UK']
df = df[~df['tmcid'].isin(values_to_remove)]
values_for_incoming = ['Incoming']
values_for_outgoing = ['Outgoing']

df_incoming = df[df['call_type'].isin(values_for_incoming)]
df_outgoing = df[df['call_type'].isin(values_for_outgoing)]
df_incoming.shape[0]
df_whoGotCutinFirstStage = df_incoming[df_incoming['tmcid'].isin(['TeleManas_Master_Inbound_DONOT_TOUCH'])]
df_whoGotCutinFirstStage.shape[0]
df_whoChoseState = df_incoming[df_incoming['tmcid'] != 'TeleManas_Master_Inbound_DONOT_TOUCH']
df_whoChoseState.shape[0]
df_whoChoseState['call - crt_object_id â†’ queue_language'].isna().sum()
mask = (df_whoChoseState['call - crt_object_id â†’ queue_language'].isna()) & (df_whoChoseState['callstatus']!='CONNECTED')
df_whoChoseLang = df_whoChoseState[~mask]
df_whoChoseLang.shape[0]

mask =  (df_whoChoseLang['callstatus']=='CONNECTED')
df_gotConnected = df_whoChoseLang[mask]
df_gotConnected.shape[0]

df_gotConnected['telemanas_id'].isnull().sum()
mask = (df_gotConnected['telemanas_id'].isnull())
df_successfulCalls = df_gotConnected[~mask]
df_successfulCalls.shape[0]
df_successfulCalls['rating'].unique()
mask = (df_successfulCalls['rating']=='1') | (df_successfulCalls['rating']=='2') | (df_successfulCalls['rating']=='3') | (df_successfulCalls['rating']=='4') | (df_successfulCalls['rating']=='5') | (df_successfulCalls['rating']=='No Input')
df_gaveRating = df_successfulCalls[mask]
df_gaveRating.shape[0]





###Important *******Trying to find the funnel stages for each state*******

import pandas as pd
import json

# Assuming df is your DataFrame
# df = pd.read_csv('your_data.csv')

# Get the unique states from 'tmcid'
states = df_whoChoseState['tmcid'].unique()

for state in states:
    # Filter dataframe for the current state
    df_choseState = df_whoChoseState[df_whoChoseState['tmcid'] == state]
    
    # Phase 1: 
    chose_state = int(df_choseState.shape[0])

    mask = (df_choseState['call - crt_object_id â†’ queue_language'].isna()) & (df_choseState['callstatus']!='CONNECTED')
    df_whoChoseLang = df_choseState[~mask]
    
    # Phase 2: 
    chose_lang = int(df_whoChoseLang.shape[0])
    
    # Phase 3: Count of rows with callstatus 'CONNECTED'
    mask =  (df_whoChoseLang['callstatus']=='CONNECTED')
    df_gotConnected = df_whoChoseLang[mask]
    connected_calls = int(df_gotConnected.shape[0])
    
    # Phase 4: Count of connected rows with non-empty telemanas_id
    mask = (df_gotConnected['telemanas_id'].isnull())
    df_successfulCalls = df_gotConnected[~mask]
    successful_calls = int(df_successfulCalls.shape[0])
    
    # Phase 5:
    mask = (df_successfulCalls['rating']=='1') | (df_successfulCalls['rating']=='2') | (df_successfulCalls['rating']=='3') | (df_successfulCalls['rating']=='4') | (df_successfulCalls['rating']=='5') | (df_successfulCalls['rating']=='No Input')
    df_gaveRating = df_successfulCalls[mask]
    gave_rating = int(df_gaveRating.shape[0])
    
    # Create the phases data as a list of dictionaries
    phases = [
        {"id": 1, "value": chose_state, "label": "Chose State"},
        {"id": 2, "value": chose_lang, "label": "Chose Language"},
        {"id": 3, "value": connected_calls, "label": "Connected Calls"},
        {"id": 4, "value": successful_calls, "label": "Successful Calls"},
        {"id": 5, "value": gave_rating, "label": "Gave Rating"}
    ]
    
    # Save the data in a file named "data_nameofthestate.js"
    filename = f"data_{state}.js"
    with open(filename, "w") as f:
        # If you need the file to be a JS file that assigns the data to a variable, uncomment the following line:
        f.write("export const data = ")
        json.dump(phases, f, indent=4)
        # If you used the above assignment, you might want to add a semicolon at the end:
        f.write(";")
    
    print(f"Data for state '{state}' written to {filename}")




import pandas as pd
import json

# Assuming df_gaveRating is your DataFrame
# df_gaveRating = pd.read_csv("your_file.csv")  # Load your data

# Group by state and rating
rating_distribution = df_gaveRating.groupby(['tmcid', 'rating']).size().unstack(fill_value=0)

# Convert counts to percentages
rating_percentages = rating_distribution.div(rating_distribution.sum(axis=1), axis=0) * 100

# Convert to dictionary
state_ratings = rating_percentages.to_dict(orient="index")

# Save JSON file
with open("rating_data.json", "w") as f:
    json.dump(state_ratings, f, indent=4)

print("Rating distribution saved to rating_data.json")


###### ***Important*******Generating for the whole country

# Count occurrences of each rating
rating_counts = df_gaveRating["rating"].value_counts().sort_index()

# Convert counts to percentage
total_ratings = rating_counts.sum()
rating_percentage = (rating_counts / total_ratings * 100).to_dict()

# Save as JSON
with open("country_rating_data.json", "w") as f:
    json.dump(rating_percentage, f, indent=4)

print("✅ Country-wide rating data saved to country_rating_data.json")


/tmp/ipykernel_51520/1183375121.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("database/Anonymized_Call_Handle_Data.csv")


KeyError: 'call - crt_object_id â†’ masked_caller_phone'

In [104]:
######output code 
import pandas as pd
import json
import os

# 1. load & basic filter
df = pd.read_csv("database/Anonymized_Call_Handle_Data.csv")

# remove unwanted tmcs
exclude_tmcs = ['ML02_TMC','docutoroutboud','KIRAN','IIITB_OB','Training_TMC_UK']
df = df[~df['tmcid'].isin(exclude_tmcs)]

# only incoming calls enter the funnel
df_incoming = df[df['call_type'] == 'Incoming']

# funnel stages as functions
def stage_chose_state(df):
    # exclude the “master inbound” bucket
    return df[df['tmcid'] != 'TeleManas_Master_Inbound_DONOT_TOUCH']

def stage_chose_language(df):
    # drop those who never selected a language (and were not connected)
    mask = (df['crt_object_id'].isna()) & (df['callstatus']!='CONNECTED')
    return df[~mask]

def stage_connected(df):
    return df[df['callstatus']=='CONNECTED']

def stage_successful(df):
    return df[df['telemanas_id'].notna()]

def stage_gave_rating(df):
    return df[df['rating'].isin(['1','2','3','4','5','No Input'])]

FUNNEL = [
    ("Received",              lambda d: d),
    ("Chose State",           stage_chose_state),
    ("Chose Language",        stage_chose_language),
    ("Connected Calls",       stage_connected),
    ("Successful Calls",      stage_successful),
    ("Gave Rating",           stage_gave_rating),
]

def compute_funnel(df):
    """
    Given an incoming‐call DataFrame, apply each funnel stage in sequence,
    record the counts, compute drop‑offs and drop‑off percentages.
    """
    labels = [lbl for lbl,_ in FUNNEL]
    values = []
    dfs = []
    current = df.copy()
    for _, fn in FUNNEL:
        current = fn(current)
        values.append(len(current))
        dfs.append(current)
    # compute dropoffs between consecutive stages
    dropoffs = [values[i] - values[i+1] for i in range(len(values)-1)]
    # percentages relative to previous stage
    dropoffPercentages = [
        round(dropoffs[i] / values[i] * 100,1) if values[i]>0 else 0.0
        for i in range(len(dropoffs))
    ]
    return {
        "labels": labels,
        "values": values,
        "dropoffs": dropoffs,
        "dropoffPercentages": dropoffPercentages
    }

# build output list
out = []

# country‑level
cf_country = compute_funnel(df_incoming)
out.append({"state":"India", "callFlow": cf_country})

# per‑state
states = stage_chose_state(df_incoming)['tmcid'].unique()
for st in states:
    df_st = df_incoming[df_incoming['tmcid']==st]
    out.append({"state": st, "callFlow": compute_funnel(df_st)})

# write JSON
os.makedirs("static", exist_ok=True)
with open("static/callflow_by_state.json","w") as f:
    json.dump(out, f, indent=2)

print("Wrote static/callflow_by_state.json")


/tmp/ipykernel_51520/2166016971.py:7: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("database/Anonymized_Call_Handle_Data.csv")


Wrote static/callflow_by_state.json


In [111]:
import pandas as pd
import json
import os

class CallFlowDashboard:
    """
    Encapsulates funnel computation for call data and writes JSON output per state and overall.
    """
    DEFAULT_EXCLUDE_TMCS = ['ML02_TMC', 'docutoroutboud', 'KIRAN', 'IIITB_OB', 'Training_TMC_UK']
    FUNNEL_STAGES = [
        ("Received", lambda df: df),
        ("Chose State", lambda df: df[df['tmcid'] != 'TeleManas_Master_Inbound_DONOT_TOUCH']),
        ("Chose Language", lambda df: df[~((df['crt_object_id'].isna()) & (df['callstatus'] != 'CONNECTED'))]),
        ("Connected Calls", lambda df: df[df['callstatus'] == 'CONNECTED']),
        ("Successful Calls", lambda df: df[df['telemanas_id'].notna()]),
        ("Gave Rating", lambda df: df[df['rating'].isin(['1','2','3','4','5','No Input'])])
    ]

    def __init__(self,  dataframe,   exclude_tmcs: list[str] = None,
                 output_dir: str = 'static',
                 output_filename: str = 'callflow_by_state.json'  , ):
        #self.csv_path = csv_path
        self.exclude_tmcs = exclude_tmcs or self.DEFAULT_EXCLUDE_TMCS
        self.output_dir = output_dir
        self.output_filename = output_filename
        self.df = dataframe
        self.df_incoming = None
        self.results: list[dict] = []



    def load_and_filter(self):
        
        self.df = self.df[~self.df['tmcid'].isin(self.exclude_tmcs)]
        self.df_incoming = self.df[self.df['call_type'] == 'Incoming']

    def compute_funnel(self, df: pd.DataFrame) -> dict:
        """
        Apply each funnel stage in sequence, record counts, dropoffs, and percentages.
        """
        labels = [label for label, _ in self.FUNNEL_STAGES]
        counts: list[int] = []
        current = df.copy()
        # apply stages
        for _, stage_fn in self.FUNNEL_STAGES:
            current = stage_fn(current)
            counts.append(len(current))
        # dropoffs between stages
        dropoffs = [counts[i] - counts[i+1] for i in range(len(counts)-1)]
        # dropoff percentages relative to previous
        dropoff_pct = [
            round((dropoffs[i] / counts[i] * 100), 1) if counts[i] > 0 else 0.0
            for i in range(len(dropoffs))
        ]
        return {
            "labels": labels,
            "values": counts,
            "dropoffs": dropoffs,
            "dropoffPercentages": dropoff_pct
        }

    def build(self,tmc=None,country=False):
        """
        Build the callFlow results for country and each state.
        """
        # country-level
        if country:
            return {
            "state": "India",
            "callFlow": self.compute_funnel(self.df_incoming)
            }
        # per-state
        # choose states after stage 2 (Chose State)

        df_state = self.df_incoming[self.df_incoming['tmcid'] == tmc]
        return {"state": tmc,"callFlow": self.compute_funnel(df_state) }
        

    def save(self):
        """
        Save the results list as JSON in the output directory.
        """
        os.makedirs(self.output_dir, exist_ok=True)
        path = os.path.join(self.output_dir, self.output_filename)
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(self.results, f, indent=2)
        print(f"Wrote {path}")

    def run(self,tmc,country):
        """
        Execute full pipeline: load, compute, and save.
        """
        print()
        self.load_and_filter()
        return self.build(tmc, country)
        #self.save()

# Usage example (uncomment in main script):
if __name__ == '__main__':
    
    dashboard = CallFlowDashboard(
         dataframe=pd.read_csv('database/Anonymized_Call_Handle_Data.csv')
     )
    print(dashboard.run(tmc= "Karnataka", country=False))


/tmp/ipykernel_51520/2403341238.py:102: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe=pd.read_csv('database/Anonymized_Call_Handle_Data.csv')



{'state': 'Karnataka', 'callFlow': {'labels': ['Received', 'Chose State', 'Chose Language', 'Connected Calls', 'Successful Calls', 'Gave Rating'], 'values': [42739, 42739, 42739, 41805, 32275, 1314], 'dropoffs': [0, 0, 934, 9530, 30961], 'dropoffPercentages': [0.0, 0.0, 2.2, 22.8, 95.9]}}


In [103]:
call = pd.read_csv("database/Anonymized_Call_Handle_Data.csv")
call.columns

/tmp/ipykernel_51520/3688443261.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  call = pd.read_csv("database/Anonymized_Call_Handle_Data.csv")


Index(['Unnamed: 0', 'crt_object_id', 'callendtime', 'callid', 'callstarttime',
       'callstatus', 'createdtime', 'tmcid', 'transferredto', 'userid',
       'customerholdtime', 'customertalktime', 'dispositioncode', 'rating',
       'call_connect', 'call_type', 'callback_required',
       'call â†’ campaign_id', 'telemanas_id', 'call â†’ incomplete_closure',
       'usertmcmapping â†’ statename', 'usertmcmapping â†’ tmcname',
       'Incomplete_closure_reason'],
      dtype='object')

In [63]:
for i, j in data.groupby('counselling_data all_complaints'):
    print(i, j.shape)

Adverse reaction after taking psychiatric medication,Excessive sleepiness,Reduced sleep,Repeated unwanted intrusive thoughts / behavior,Sadness of mood,Talking excessively,Violence/Aggression towards others (1, 21)
Adverse reaction after taking psychiatric medication,Feeling tense/stressed most of the time on all days for most of the time (1, 21)
Adverse reaction after taking psychiatric medication,Increased use of an illicit substance (alcohol, tobacco and others) (Harmful/Hazardous use) (1, 21)
Adverse reaction after taking psychiatric medication,Reduced sleep (1, 21)
Anxiety,Medical Issues,Sadness of mood,Sleep Disturbances (5, 21)
Behavioral disturbance due to excessive substance use or use of gadgets/ social media,Excessive use of technological gadgets/social media,Feeling anxious/fearful in specific situations,Low energy/Fatigue,Reduced interaction with others,Reduced self-care (1, 21)
Behavioral disturbance due to excessive substance use or use of gadgets/ social media,Hearing v

In [112]:
import pandas as pd
import json
import os

class CallFlowDashboard:
    """
    Encapsulates funnel computation for call data and writes JSON output per state and overall.
    """
    DEFAULT_EXCLUDE_TMCS = ['ML02_TMC', 'docutoroutboud', 'KIRAN', 'IIITB_OB', 'Training_TMC_UK']
    FUNNEL_STAGES = [
        ("Received", lambda df: df),
        ("Chose State", lambda df: df[df['tmcid'] != 'TeleManas_Master_Inbound_DONOT_TOUCH']),
        ("Chose Language", lambda df: df[~((df['crt_object_id'].isna()) & (df['callstatus'] != 'CONNECTED'))]),
        ("Connected Calls", lambda df: df[df['callstatus'] == 'CONNECTED']),
        ("Successful Calls", lambda df: df[df['telemanas_id'].notna()]),
        ("Gave Rating", lambda df: df[df['rating'].isin(['1','2','3','4','5','No Input'])])
    ]

    def __init__(self,  dataframe,   exclude_tmcs: list[str] = None,
                 output_dir: str = 'static',
                 output_filename: str = 'callflow_by_state.json'  , ):
        #self.csv_path = csv_path
        self.exclude_tmcs = exclude_tmcs or self.DEFAULT_EXCLUDE_TMCS
        self.output_dir = output_dir
        self.output_filename = output_filename
        self.df = dataframe
        self.df_incoming = None
        self.results: list[dict] = []



    def load_and_filter(self):
        
        self.df = self.df[~self.df['tmcid'].isin(self.exclude_tmcs)]
        self.df_incoming = self.df[self.df['call_type'] == 'Incoming']

    def compute_funnel(self, df: pd.DataFrame) -> dict:
        """
        Apply each funnel stage in sequence, record counts, dropoffs, and percentages.
        """
        labels = [label for label, _ in self.FUNNEL_STAGES]
        counts: list[int] = []
        current = df.copy()
        # apply stages
        for _, stage_fn in self.FUNNEL_STAGES:
            current = stage_fn(current)
            counts.append(len(current))
        # dropoffs between stages
        dropoffs = [counts[i] - counts[i+1] for i in range(len(counts)-1)]
        # dropoff percentages relative to previous
        dropoff_pct = [
            round((dropoffs[i] / counts[i] * 100), 1) if counts[i] > 0 else 0.0
            for i in range(len(dropoffs))
        ]
        return {
            "labels": labels,
            "values": counts,
            "dropoffs": dropoffs,
            "dropoffPercentages": dropoff_pct
        }

    def build(self,tmc=None,country=False):
        """
        Build the callFlow results for country and each state.
        """
        # country-level
        if country:
            return {
            "state": "India",
            "callFlow": self.compute_funnel(self.df_incoming)
            }
        # per-state
        # choose states after stage 2 (Chose State)

        df_state = self.df_incoming[self.df_incoming['tmcid'] == tmc]
        return {"state": tmc,"callFlow": self.compute_funnel(df_state) }
        

    def save(self):
        """
        Save the results list as JSON in the output directory.
        """
        os.makedirs(self.output_dir, exist_ok=True)
        path = os.path.join(self.output_dir, self.output_filename)
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(self.results, f, indent=2)
        print(f"Wrote {path}")

    def run(self,tmc,country):
        """
        Execute full pipeline: load, compute, and save.
        """
        print()
        self.load_and_filter()
        return self.build(tmc, country)

















class extract_for_dashboard:

    def __init__(self, call_analysis_path,funnel_chart_dataset):
        self.df = pd.read_csv(call_analysis_path)

        # keeping only rows with valid genders
        keep = {'Male', 'Female', 'Transgender'}
        self.df = self.df[self.df['Gender'].isin(keep)].reset_index(drop=True)

        # now prepare the data
        self.prepare_data_before_moving_on()
        self.final_json = []

        self.funnel_data =  pd.read_csv(funnel_chart_dataset)
        self.object_for_funnel_chart = CallFlowDashboard(self.funnel_data)












    def prepare_data_before_moving_on(self):
        self.df['callendtime'] = pd.to_datetime(self.df['callendtime'], errors='coerce')
        self.df['callstarttime'] = pd.to_datetime(self.df['callstarttime'], errors='coerce')

        # Drop rows with invalid datetimes
        self.df = self.df.dropna(subset=['callendtime', 'callstarttime'])

        # Extract date
        self.df['callenddate'] = pd.to_datetime(self.df['callendtime'].dt.date)

        # Extract time from datetime
        self.df['callendtimeframe'] = self.df['callendtime'].dt.time
        self.df['callstarttimeframe'] = self.df['callstarttime'].dt.time

        def time_to_minutes(t):
            return t.hour * 60 + t.minute + t.second / 60

        self.df['callend_minutes'] = self.df['callendtimeframe'].apply(time_to_minutes)
        self.df['callstart_minutes'] = self.df['callstarttimeframe'].apply(time_to_minutes)

        self.df['call_duration_minutes_from_time'] = self.df['callend_minutes'] - self.df['callstart_minutes']
        self.df['day'] = self.df['callenddate'].dt.day_name()
        self.df['Month'] = self.df['callenddate'].dt.strftime('%b')


        midnight = self.df['customertalktime'].dt.normalize()
        # subtract to get a Timedelta (duration since midnight)
        self.df['customertalktime'] = self.df['customertalktime'] - midnight


    def gender_count(self, df):
        temp = {i: j.shape[0] for i, j in df.groupby('Gender')}
        for i in ['Male', 'Female', 'Transgender']:
            if i not in temp:
                temp[i] = 0
        return temp

    def timeseries(self, df):
        return [{"date": str(i), "calls": j.shape[0]} for i, j in df.groupby('callenddate')]

    def avg_duration(self, df):
        return [
        {
        "date": date.strftime('%Y-%m-%d'),  # Format the date without time
        "minutes": round(group['customertalktime'].mean().total_seconds() / 60, 2)
        }
        for date, group in d1.groupby('callenddate')
        ]
    

    def byWeekday(self, df):
        return {i[:3]: j.shape[0] for i, j in df.groupby('day')}

    def byAgeGroup(self, dataframe):
        age_col = "patient â†’ age"
        df = dataframe.dropna(subset=[age_col])
        df[age_col] = pd.to_numeric(df[age_col], errors='coerce')
        df = df.dropna(subset=[age_col])

        bins = [0, 18, 25, 35, 45, 55, float('inf')]
        labels = ["Under 18", "18-24", "25-34", "35-44", "45-54", "55+"]
        df['age_group'] = pd.cut(df[age_col], bins=bins, labels=labels, right=False)
        age_counts = df['age_group'].value_counts().reindex(labels, fill_value=0)
        return age_counts.to_dict()

    def give_call_count(self, dataframe, month):
        temp = {'month': month}
        for i, j in dataframe.groupby('call_type'):
            temp[i.lower()] = j.shape[0]
        return temp

    def callsByDirection(self, dataframe):
        return [self.give_call_count(j, i) for i, j in dataframe.groupby('Month')]

    def triage(self, dataframe):
        return {i: j.shape[0] for i, j in dataframe.groupby('triage')}

    def callflow(self, df):
        return 10  # Placeholder

    def compute_for_india(self):
        temp = {
            'state': "India",
            "totalCalls": self.df.shape[0],
            "byGender": self.gender_count(self.df),
            "timeseries": self.timeseries(self.df),
            "avgDuration": self.avg_duration(self.df),
            'byWeekday': self.byWeekday(self.df),
            'byAgeGroup': self.byAgeGroup(self.df),
            'callsByDirection': self.callsByDirection(self.df),
            'triage': self.triage(self.df),
            'callflow': self.object_for_funnel_chart.run(tmc="India",country=True),
            
        }
        self.final_json.append(temp)

    def store(self, filename):
        print("storing the data for dashboard")
        if not os.path.exists('static'):
            os.makedirs('static')
        file_path = os.path.join("static", filename)
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(self.final_json, f, ensure_ascii=False, indent=2)

    def index(self):
        self.compute_for_india()

        for i in self.df['tmcid'].unique():
            temp_df_state = self.df[self.df['tmcid'] == i]
            temp = {
                "state": i,
                "totalCalls": temp_df_state.shape[0],
                "byGender": self.gender_count(temp_df_state),
                "timeseries": self.timeseries(temp_df_state),
                "avgDuration": self.avg_duration(temp_df_state),
                "byWeekday": self.byWeekday(temp_df_state),
                "byAgeGroup": self.byAgeGroup(temp_df_state),
                "callsByDirection": self.callsByDirection(temp_df_state),
                "triage": self.triage(temp_df_state),
                "callflow": self.object_for_funnel_chart.run(tmc= i, country=False),
            }
            self.final_json.append(temp)

        # ✅ Fixed: use filename instead of the final_json list
        self.store("states.json")




extract_for_dashboard(call_analysis_path="database/counselling_data.csv",funnel_chart_dataset="database/Anonymized_Call_Handle_Data.csv").index()

/tmp/ipykernel_51520/2772667753.py:116: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(call_analysis_path)


AttributeError: Can only use .dt accessor with datetimelike values

In [115]:
import pandas as pd
import json
import os

class CallFlowDashboard:
    """
    Encapsulates funnel computation for call data.
    """
    DEFAULT_EXCLUDE_TMCS = ['ML02_TMC', 'docutoroutboud', 'KIRAN', 'IIITB_OB', 'Training_TMC_UK']
    FUNNEL_STAGES = [
        ("Received",       lambda df: df),
        ("Chose State",    lambda df: df[df['tmcid'] != 'TeleManas_Master_Inbound_DONOT_TOUCH']),
        ("Chose Language", lambda df: df[~((df['crt_object_id'].isna()) & (df['callstatus'] != 'CONNECTED'))]),
        ("Connected",      lambda df: df[df['callstatus'] == 'CONNECTED']),
        ("Successful",     lambda df: df[df['telemanas_id'].notna()]),
        ("Gave Rating",    lambda df: df[df['rating'].isin(['1','2','3','4','5','No Input'])])
    ]

    def __init__(self, df: pd.DataFrame, exclude_tmcs=None):
        self.exclude_tmcs = exclude_tmcs or self.DEFAULT_EXCLUDE_TMCS
        self.df = df.copy()
        self.df_in = None

    def load_and_filter(self):
        # remove unwanted tmcs, keep only Incoming
        self.df = self.df[~self.df['tmcid'].isin(self.exclude_tmcs)]
        self.df_in = self.df[self.df['call_type'] == 'Incoming']

    def compute_funnel(self, df: pd.DataFrame) -> dict:
        labels = [lbl for lbl,_ in self.FUNNEL_STAGES]
        counts = []
        cur = df
        for _, fn in self.FUNNEL_STAGES:
            cur = fn(cur)
            counts.append(len(cur))
        dropoffs = [counts[i] - counts[i+1] for i in range(len(counts)-1)]
        dropoffPct = [ round(dropoffs[i]/counts[i]*100,1) if counts[i]>0 else 0.0
                       for i in range(len(dropoffs)) ]
        return {
            "labels": labels,
            "values": counts,
            "dropoffs": dropoffs,
            "dropoffPercentages": dropoffPct
        }

    def run(self):
        self.load_and_filter()
        out = []
        # country-level
        out.append({"state":"India", "callFlow": self.compute_funnel(self.df_in)})
        # state-level
        after_state = self.FUNNEL_STAGES[1][1](self.df_in)  # after "Chose State"
        for st in after_state['tmcid'].unique():
            df_st = self.df_in[self.df_in['tmcid']==st]
            out.append({"state": st, "callFlow": self.compute_funnel(df_st)})
        return out


class extract_for_dashboard:
    def __init__(self, call_analysis_path, funnel_chart_dataset):
        # load main call data
        self.df = pd.read_csv(call_analysis_path)
        # filter genders
        self.df = self.df[self.df['Gender'].isin(['Male','Female','Transgender'])]
        # load funnel dataset
        self.funnel_df = pd.read_csv(funnel_chart_dataset)
        self.final_json = []

    def prepare_data_before_moving_on(self):
        df = self.df
        # parse times
        df['callstarttime'] = pd.to_datetime(df['callstarttime'], errors='coerce')
        df['callendtime']   = pd.to_datetime(df['callendtime'],   errors='coerce')
        df = df.dropna(subset=['callstarttime','callendtime'])
        df['callenddate'] = df['callendtime'].dt.date
        df['day']        = df['callenddate'].apply(lambda d: pd.to_datetime(d).day_name())
        df['Month']      = df['callenddate'].apply(lambda d: d.strftime('%b'))
        # customer talk time: convert then normalize
        df['customertalktime'] = pd.to_datetime(df['customertalktime'], errors='coerce')
        df = df.dropna(subset=['customertalktime'])
        midnight = df['customertalktime'].dt.normalize()
        df['customertalktime'] = df['customertalktime'] - midnight
        self.df = df

    def avg_duration(self, df):
        # average of customertalktime per day
        out = []
        for date, grp in df.groupby('callenddate'):
            mins = grp['customertalktime'].dt.total_seconds().mean() / 60
            out.append({"date": str(date), "minutes": round(mins,2)})
        return out

    def gender_count(self, df):
        cnt = df['Gender'].value_counts().to_dict()
        for g in ['Male','Female','Transgender']:
            cnt.setdefault(g,0)
        return cnt

    def timeseries(self, df):
        return [{"date":str(d), "calls":len(g)} for d,g in df.groupby('callenddate')]

    def byWeekday(self, df):
        return {day[:3]:len(g) for day,g in df.groupby('day')}

    def byAgeGroup(self, df):
        col = "patient â†’ age"
        df2 = df.dropna(subset=[col])
        df2[col] = pd.to_numeric(df2[col], errors='coerce')
        df2 = df2.dropna(subset=[col])
        bins = [0,18,25,35,45,55,float('inf')]
        labels = ["Under 18","18-24","25-34","35-44","45-54","55+"]
        grp = pd.cut(df2[col], bins=bins, labels=labels, right=False)
        return grp.value_counts().reindex(labels, fill_value=0).to_dict()

    def callsByDirection(self, df):
        out = []
        for m,grp in df.groupby('Month'):
            d = {"month":m}
            d.update(grp['call_type'].value_counts().to_dict())
            out.append(d)
        return out

    def triage(self, df):
        return df['triage'].value_counts().to_dict()

    def index(self):
        # preprocess
        self.prepare_data_before_moving_on()
        # funnel object
        funneler = CallFlowDashboard(self.funnel_df)
        funnel_out = funneler.run()

        # India summary
        india = {
            "state":"India",
            "totalCalls": len(self.df),
            "byGender": self.gender_count(self.df),
            "timeseries": self.timeseries(self.df),
            "avgDuration": self.avg_duration(self.df),
            "byWeekday": self.byWeekday(self.df),
            "byAgeGroup": self.byAgeGroup(self.df),
            "callsByDirection": self.callsByDirection(self.df),
            "triage": self.triage(self.df),
            "callflow": next(x for x in funnel_out if x['state']=="India")['callFlow']
        }
        self.final_json.append(india)

        # per-state
        for state in self.df['tmcid'].unique():
            df_st = self.df[self.df['tmcid']==state]
            self.final_json.append({
                "state": state,
                "totalCalls": len(df_st),
                "byGender": self.gender_count(df_st),
                "timeseries": self.timeseries(df_st),
                "avgDuration": self.avg_duration(df_st),
                "byWeekday": self.byWeekday(df_st),
                "byAgeGroup": self.byAgeGroup(df_st),
                "callsByDirection": self.callsByDirection(df_st),
                "triage": self.triage(df_st),
                "callflow": next(x for x in funnel_out if x['state']==state)["callFlow"]
            })

        # store
        os.makedirs("static", exist_ok=True)
        with open("static/states.json","w") as f:
            json.dump(self.final_json, f, indent=2)

        return self.final_json


# Example usage:
extractor = extract_for_dashboard(
     call_analysis_path="database/counselling_data.csv",
   funnel_chart_dataset="database/Anonymized_Call_Handle_Data.csv"
 )
data = extractor.index()
#print(json.dumps(data, indent=2))


/tmp/ipykernel_51520/263384465.py:62: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(call_analysis_path)
/tmp/ipykernel_51520/263384465.py:66: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  self.funnel_df = pd.read_csv(funnel_chart_dataset)
/tmp/ipykernel_51520/263384465.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['callenddate'] = df['callendtime'].dt.date
/tmp/ipykernel_51520/263384465.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

[
  {
    "state": "India",
    "totalCalls": 142558,
    "byGender": {
      "Male": 82654,
      "Female": 59885,
      "Transgender": 19
    },
    "timeseries": [
      {
        "date": "2024-10-04",
        "calls": 1681
      },
      {
        "date": "2024-10-05",
        "calls": 1651
      },
      {
        "date": "2024-10-06",
        "calls": 1265
      },
      {
        "date": "2024-10-07",
        "calls": 1922
      },
      {
        "date": "2024-10-08",
        "calls": 2215
      },
      {
        "date": "2024-10-09",
        "calls": 1896
      },
      {
        "date": "2024-10-10",
        "calls": 1605
      },
      {
        "date": "2024-10-11",
        "calls": 1584
      },
      {
        "date": "2024-10-12",
        "calls": 1432
      },
      {
        "date": "2024-10-13",
        "calls": 1240
      },
      {
        "date": "2024-10-14",
        "calls": 1971
      },
      {
        "date": "2024-10-15",
        "calls": 2008
      },
     

In [95]:
import pandas as pd
import json
import os

class extract_for_dashboard1:

    def __init__(self, call_analysis_path):
        self.df = pd.read_csv(call_analysis_path)

        # keeping only rows with valid genders
        keep = {'Male', 'Female', 'Transgender'}
        self.df = self.df[self.df['Gender'].isin(keep)].reset_index(drop=True)

        # now prepare the data
        self.prepare_data_before_moving_on()

        self.final_json = []

    def prepare_data_before_moving_on(self):
        self.df['callendtime'] = pd.to_datetime(self.df['callendtime'], errors='coerce')
        self.df['callstarttime'] = pd.to_datetime(self.df['callstarttime'], errors='coerce')

        # Drop rows with invalid datetimes
        self.df = self.df.dropna(subset=['callendtime', 'callstarttime'])

        # Extract date
        self.df['callenddate'] = pd.to_datetime(self.df['callendtime'].dt.date)

        # Extract time from datetime
        self.df['callendtimeframe'] = self.df['callendtime'].dt.time
        self.df['callstarttimeframe'] = self.df['callstarttime'].dt.time

        def time_to_minutes(t):
            return t.hour * 60 + t.minute + t.second / 60

        self.df['callend_minutes'] = self.df['callendtimeframe'].apply(time_to_minutes)
        self.df['callstart_minutes'] = self.df['callstarttimeframe'].apply(time_to_minutes)

        self.df['call_duration_minutes_from_time'] = self.df['callend_minutes'] - self.df['callstart_minutes']
        self.df['day'] = self.df['callenddate'].dt.day_name()
        self.df['Month'] = self.df['callenddate'].dt.strftime('%b')

        # ✅ Convert customertalktime to timedelta
        self.df['customertalktime'] = pd.to_timedelta(self.df['customertalktime'], errors='coerce')

    def gender_count(self, df):
        temp = {i: j.shape[0] for i, j in df.groupby('Gender')}
        for i in ['Male', 'Female', 'Transgender']:
            if i not in temp:
                temp[i] = 0
        return temp

    def timeseries(self, df):
        return [{"date": str(i), "calls": j.shape[0]} for i, j in df.groupby('callenddate')]

    def avg_duration(self, df):
        return [
            {
                "date": date.strftime('%Y-%m-%d'),
                "minutes": round(group['customertalktime'].mean().total_seconds() / 60, 2)
            }
            for date, group in df.groupby('callenddate')
            if not group['customertalktime'].isna().all()
        ]

    def byWeekday(self, df):
        return {i[:3]: j.shape[0] for i, j in df.groupby('day')}

    def byAgeGroup(self, dataframe):
        age_col = "patient â†’ age"
        df = dataframe.dropna(subset=[age_col])
        df[age_col] = pd.to_numeric(df[age_col], errors='coerce')
        df = df.dropna(subset=[age_col])

        bins = [0, 18, 25, 35, 45, 55, float('inf')]
        labels = ["Under 18", "18-24", "25-34", "35-44", "45-54", "55+"]
        df['age_group'] = pd.cut(df[age_col], bins=bins, labels=labels, right=False)
        age_counts = df['age_group'].value_counts().reindex(labels, fill_value=0)
        return age_counts.to_dict()

    def give_call_count(self, dataframe, month):
        temp = {'month': month}
        for i, j in dataframe.groupby('call_type'):
            temp[i.lower()] = j.shape[0]
        return temp

    def callsByDirection(self, dataframe):
        return [self.give_call_count(j, i) for i, j in dataframe.groupby('Month')]

    def triage(self, dataframe):
        return {i: j.shape[0] for i, j in dataframe.groupby('triage')}

    def callflow(self, df):
        return 10  # Placeholder

    def compute_for_india(self):
        temp = {
            'state': "India",
            "totalCalls": self.df.shape[0],
            "byGender": self.gender_count(self.df),
            "timeseries": self.timeseries(self.df),
            "avgDuration": self.avg_duration(self.df),
            'byWeekday': self.byWeekday(self.df),
            'byAgeGroup': self.byAgeGroup(self.df),
            'callsByDirection': self.callsByDirection(self.df),
            'triage': self.triage(self.df),
            'callflow': self.callflow(self.df),
        }
        self.final_json.append(temp)

    def store(self, filename):
        print("storing the data for dashboard")
        if not os.path.exists('static'):
            os.makedirs('static')
        file_path = os.path.join("static", filename)
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(self.final_json, f, ensure_ascii=False, indent=2)

    def index(self):
        self.compute_for_india()

        for i in self.df['tmcid'].unique():
            temp_df_state = self.df[self.df['tmcid'] == i]
            temp = {
                "state": i,
                "totalCalls": temp_df_state.shape[0],
                "byGender": self.gender_count(temp_df_state),
                "timeseries": self.timeseries(temp_df_state),
                "avgDuration": self.avg_duration(temp_df_state),
                "byWeekday": self.byWeekday(temp_df_state),
                "byAgeGroup": self.byAgeGroup(temp_df_state),
                "callsByDirection": self.callsByDirection(temp_df_state),
                "triage": self.triage(temp_df_state),
                "callflow": self.callflow(temp_df_state),
            }
            self.final_json.append(temp)

        self.store("states.json")


In [98]:
import pandas as pd
import json
import os

class extract_for_dashboard2:

    def __init__(self, call_analysis_path):
        self.df = pd.read_csv(call_analysis_path)
        self.df = self.df[self.df['Gender'].isin({'Male', 'Female', 'Transgender'})].reset_index(drop=True)
        self.final_json = []
        self.prepare_data()

    def prepare_data(self):
        df = self.df

        # Preprocess datetime columns
        df['callendtime'] = pd.to_datetime(df['callendtime'], errors='coerce')
        df['callstarttime'] = pd.to_datetime(df['callstarttime'], errors='coerce')
        df.dropna(subset=['callendtime', 'callstarttime'], inplace=True)

        df['callenddate'] = df['callendtime'].dt.normalize()
        df['day'] = df['callendtime'].dt.day_name()
        df['Month'] = df['callendtime'].dt.strftime('%b')
        df['customertalktime'] = pd.to_timedelta(df['customertalktime'], errors='coerce')

        # Optional: remove unnecessary columns
        self.df = df

    def gender_count(self, df):
        counts = df['Gender'].value_counts().to_dict()
        return {g: counts.get(g, 0) for g in ['Male', 'Female', 'Transgender']}

    def timeseries(self, df):
        counts = df['callenddate'].value_counts().sort_index()
        return [{"date": str(date.date()), "calls": count} for date, count in counts.items()]

    def avg_duration(self, df):
        grouped = df[['callenddate', 'customertalktime']].dropna().groupby('callenddate')['customertalktime']
        return [{"date": str(date.date()), "minutes": round(duration.mean().total_seconds() / 60, 2)}
                for date, duration in grouped]

    def byWeekday(self, df):
        return df['day'].value_counts().sort_index().to_dict()

    def byAgeGroup(self, df):
        age_col = "patient â†’ age"
        if age_col not in df.columns:
            return {}
        df = df.copy()
        df[age_col] = pd.to_numeric(df[age_col], errors='coerce')
        df = df.dropna(subset=[age_col])
        bins = [0, 18, 25, 35, 45, 55, float('inf')]
        labels = ["Under 18", "18-24", "25-34", "35-44", "45-54", "55+"]
        df['age_group'] = pd.cut(df[age_col], bins=bins, labels=labels, right=False)
        return df['age_group'].value_counts().reindex(labels, fill_value=0).to_dict()

    def callsByDirection(self, df):
        result = []
        grouped = df.groupby(['Month', 'call_type']).size().unstack(fill_value=0)
        for month in grouped.index:
            month_data = {'month': month}
            for call_type in grouped.columns:
                month_data[call_type.lower()] = int(grouped.loc[month, call_type])
            result.append(month_data)
        return result

    def triage(self, df):
        return df['triage'].value_counts().to_dict()

    def callflow(self, df):
        return 10  # Placeholder

    def compute_metrics(self, df, state_name):
        return {
            'state': state_name,
            'totalCalls': df.shape[0],
            'byGender': self.gender_count(df),
            'timeseries': self.timeseries(df),
            'avgDuration': self.avg_duration(df),
            'byWeekday': self.byWeekday(df),
            'byAgeGroup': self.byAgeGroup(df),
            'callsByDirection': self.callsByDirection(df),
            'triage': self.triage(df),
            'callflow': self.callflow(df),
        }

    def store(self, filename):
        print("Storing the data for dashboard")
        os.makedirs("static", exist_ok=True)
        with open(os.path.join("static", filename), 'w', encoding='utf-8') as f:
            json.dump(self.final_json, f, ensure_ascii=False, indent=2)

    def index(self):
        # Compute for India (entire dataset)
        self.final_json.append(self.compute_metrics(self.df, "India"))

        # Compute per state
        for tmcid, group in self.df.groupby("tmcid"):
            self.final_json.append(self.compute_metrics(group, tmcid))

        self.store("states.json")


In [99]:
extract_for_dashboard2("database/counselling_data.csv").index()

/tmp/ipykernel_51520/2152694540.py:8: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(call_analysis_path)


Storing the data for dashboard
